### Load case reports

In [3]:
import pandas as pd
filtered_df = pd.read_csv('data/filtered_case_reports.csv')


### Generate case report pairs

In [5]:
from itertools import combinations

# Function to calculate the score based on shared keywords
def calculate_score(keywords1, keywords2):
    return len(set(keywords1) & set(keywords2))

# Generate pairs of abstracts with their scores
abstract_pairs = []
for (idx1, row1), (idx2, row2) in combinations(filtered_df.iterrows(), 2):
    score = calculate_score(row1['processed_keywords'], row2['processed_keywords'])
    abstract_pairs.append((row1['abstract'], row2['abstract'], score, row1['processed_keywords'], row2['processed_keywords']))


#count pairs by score
from collections import Counter
score_counter = Counter([score for _, _, score, _, _ in abstract_pairs])
score_counter.most_common(10)

[(0, 145727), (1, 833), (2, 46), (3, 4), (4, 1)]

In [6]:
len(abstract_pairs)

146611

### Balance similar and non-similar pairs

In [7]:
#balance pairs by score
from sklearn.utils import resample

# Split the data into majority and minority classes
majority_class = [pair for pair in abstract_pairs if pair[2] == 0]
minority_class = [pair for pair in abstract_pairs if pair[2] > 0]

# Undersample the majority class to match the size of the minority class
undersampled_majority_class = resample(majority_class, 
                                       replace=False,     # Don't duplicate samples
                                       n_samples= len(minority_class),  # Match minority class size
                                       random_state=42)

# Combine undersampled majority class with the minority class
balanced_abstract_pairs = undersampled_majority_class + minority_class

# count pairs by score
score_counter = Counter([score for _, _, score, _, _ in balanced_abstract_pairs])
score_counter.most_common(10)

[(0, 884), (1, 833), (2, 46), (3, 4), (4, 1)]

### Similarity score based on the number of common keywords

In [8]:
def normalize_score(score):
    if score == 0:
        return 0
    elif score == 1:
        return 0.65
    elif score == 2:
        return 0.85
    elif score == 3:
        return 0.95
    elif score == 4:
        return 1

# Apply the normalization function
balanced_abstract_pairs = [(abstract1, abstract2, normalize_score(score), keywords1, keywords2)
                  for abstract1, abstract2, score, keywords1, keywords2 in balanced_abstract_pairs]

In [9]:
# sort by score
balanced_abstract_pairs = sorted(balanced_abstract_pairs, key=lambda x: x[2], reverse=True)
# print pairs
for abstract1, abstract2, score, k1, k2 in balanced_abstract_pairs[:5]:
    print(f"Score: {score}")
    print(f"Abstract 1: {abstract1}")
    print(f"Abstract 2: {abstract2}")
    print(f"Keywords 1: {k1}")
    print(f"Keywords 2: {k2}")
    print("-" * 80)

Score: 1
Abstract 1: A Doença de Still do Adulto (DSA) é uma doença inflamatória sistémica, rara, de etiologia desconhecida, cujas principais manifestações incluem febre elevada, exantema maculopapular evanescente, artralgias/artrite e odinofagia persistente. O diagnóstico é clínico e implica a exclusão de outras patologias infeciosas, autoimunes e neoplásicas. Os autores apresentam o caso clínico de doente de 22 anos, previamente saudável, com quadro com cerca de 5 meses de evolução, caraterizado por poliartralgias simétricas de ritmo inflamatório, com incapacidade funcional marcada, associadas a febre vespertina, exantema evanescente, astenia, anorexia e perda ponderal. Ao exame físico apresentava mucosas descoradas, sinovite das articulações metacarpofalangicas e interfalangicas proximais, e múltiplas adenopatias infra e pericentimétricas, não dolorosas, móveis, de consistência duro-elástica, cervicais, axilares e inguinais. Analiticamente com elevação marcada da ferritina, com fraç

### Generate test and train split

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np


filtered_balanced_abstract_pairs = [(abstract1, abstract2, score) for abstract1, abstract2, score, _, _ in balanced_abstract_pairs]
df_ = pd.DataFrame(filtered_balanced_abstract_pairs, columns=['abstract1', 'abstract2', 'score'])


bins = np.arange(-0.1, 1.1, 0.1)  # Creates bins from 0.0 to 1.0 with a width of 0.1
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins) - 1)]


df_['score_bin'] = pd.cut(df_['score'], bins=bins, labels=labels, right=True)

train_df, test_df = train_test_split(
    df_,
    test_size=0.2,  # 20% for testing
    stratify=df_['score_bin'],  # Stratify based on the score bins
    random_state=42  # For reproducibility
)

train_data = list(zip(train_df['abstract1'], train_df['abstract2'], train_df['score']))
test_data = list(zip(test_df['abstract1'], test_df['abstract2'], test_df['score']))

#print counts of entries by score
print(train_df['score_bin'].value_counts())
print(test_df['score_bin'].value_counts())

score_bin
-0.1-0.0                                   707
0.6000000000000001-0.7000000000000001      666
0.8-0.9                                     37
0.9-1.0                                      4
0.0-0.1                                      0
0.1-0.20000000000000004                      0
0.20000000000000004-0.30000000000000004      0
0.30000000000000004-0.4                      0
0.4-0.5000000000000001                       0
0.5000000000000001-0.6000000000000001        0
0.7000000000000001-0.8                       0
1.0-1.1                                      0
Name: count, dtype: int64
score_bin
-0.1-0.0                                   177
0.6000000000000001-0.7000000000000001      167
0.8-0.9                                      9
0.9-1.0                                      1
0.0-0.1                                      0
0.1-0.20000000000000004                      0
0.20000000000000004-0.30000000000000004      0
0.30000000000000004-0.4                      0
0.4-0.50000000

In [10]:
#converto to hugginface dataset
from datasets import Dataset
from datasets import DatasetDict
train_dataset = Dataset.from_pandas(pd.DataFrame(train_data, columns=["abstract1", "abstract2", "score"]))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_data, columns=["abstract1", "abstract2", "score"]))



### Train the Bi-encoder model

In [ ]:
from sentence_transformers import SentenceTransformer, losses


#model1 = SentenceTransformer('distiluse-base-multilingual-cased-v1')# 
model1 = SentenceTransformer('neuralmind/bert-base-portuguese-cased')
#model1 = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
loss1= losses.CosineSimilarityLoss(model1)

c:\Users\Filip\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.similarity_functions import SimilarityFunction
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator



# 5. (Optional) Specify training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/sentence_transformers/bert-base-portuguese",
    # Optional training parameters:
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_steps=100,
    run_name="sentence_transformers_bert-base-portuguese",  # Will be used in W&B if `wandb` is installed
)


# Create the evaluator
evaluator = EmbeddingSimilarityEvaluator(
    test_dataset['abstract1'],  # Assuming these are the sentence pairs for evaluation
    test_dataset['abstract2'],
    test_dataset['score'],  # Assuming this contains the similarity scores
    main_similarity=SimilarityFunction.COSINE,
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model1,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss=loss1,
    evaluator=evaluator,
)



In [15]:
trainer.train()

  5%|▌         | 100/1930 [00:47<14:17,  2.13it/s]

{'loss': 0.1076, 'grad_norm': 1.9443687200546265, 'learning_rate': 2.5906735751295337e-05, 'epoch': 0.52}



  5%|▌         | 100/1930 [00:56<14:17,  2.13it/s]

{'eval_loss': 0.12303422391414642, 'eval_pearson_cosine': 0.3108885805349722, 'eval_spearman_cosine': 0.31321639860465283, 'eval_pearson_manhattan': 0.28447131285553723, 'eval_spearman_manhattan': 0.2908359018332696, 'eval_pearson_euclidean': 0.28732107522405664, 'eval_spearman_euclidean': 0.2924603646297584, 'eval_pearson_dot': 0.29768883204450025, 'eval_spearman_dot': 0.30908777855913117, 'eval_pearson_max': 0.3108885805349722, 'eval_spearman_max': 0.31321639860465283, 'eval_runtime': 9.4749, 'eval_samples_per_second': 40.739, 'eval_steps_per_second': 5.172, 'epoch': 0.52}


 10%|█         | 200/1930 [01:57<13:22,  2.16it/s]  

...

                                                   
 98%|█████████▊| 1900/1930 [43:19<00:13,  2.15it/s]

{'eval_loss': 0.06656013429164886, 'eval_pearson_cosine': 0.6352652863839424, 'eval_spearman_cosine': 0.6401309474893349, 'eval_pearson_manhattan': 0.6134625003830313, 'eval_spearman_manhattan': 0.6191032781328102, 'eval_pearson_euclidean': 0.6154400849864002, 'eval_spearman_euclidean': 0.6236359632383587, 'eval_pearson_dot': 0.6253966365594591, 'eval_spearman_dot': 0.6313444641952204, 'eval_pearson_max': 0.6352652863839424, 'eval_spearman_max': 0.6401309474893349, 'eval_runtime': 9.4628, 'eval_samples_per_second': 40.791, 'eval_steps_per_second': 5.178, 'epoch': 9.84}


100%|██████████| 1930/1930 [45:44<00:00,  1.42s/it]

{'train_runtime': 2744.0541, 'train_samples_per_second': 5.612, 'train_steps_per_second': 0.703, 'train_loss': 0.025580150809698773, 'epoch': 10.0}


TrainOutput(global_step=1930, training_loss=0.025580150809698773, metrics={'train_runtime': 2744.0541, 'train_samples_per_second': 5.612, 'train_steps_per_second': 0.703, 'total_flos': 0.0, 'train_loss': 0.025580150809698773, 'epoch': 10.0})

### Test bi-encoder model

In [16]:



# 7. Evaluate the model performance on the STS Benchmark test dataset
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["abstract1"],
    sentences2=test_dataset["abstract2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-test",
)
test_evaluator(model1)


{'sts-test_pearson_cosine': 0.6352652863839424,
 'sts-test_spearman_cosine': 0.6401309474893349,
 'sts-test_pearson_manhattan': 0.6134625003830313,
 'sts-test_spearman_manhattan': 0.6191032781328102,
 'sts-test_pearson_euclidean': 0.6154400849864002,
 'sts-test_spearman_euclidean': 0.6236359632383587,
 'sts-test_pearson_dot': 0.6253966365594591,
 'sts-test_spearman_dot': 0.6313444641952204,
 'sts-test_pearson_max': 0.6352652863839424,
 'sts-test_spearman_max': 0.6401309474893349}

In [ ]:


# 8. Save the trained model
model1.save_pretrained("directory_path")